In [1]:
import pandas as pd
hospital_data = pd.read_csv(r"E:datasets\Hospital_Directory.csv")
hospital_data

,X,Y,objectid,health_facility_name,address,street,landmark,locality,posatalcode,landline_number,facility_type,state_name,district_name,taluka_name,block_name,lat,lon,altitude
0,78.564177,12.492635,1,THIRUPATHUR UPHC,1 THIRUPATHUR UPHC,Chairman Laxshmanan Street,"Tirupattur Municipal Office, Opp. DDHS tirupattur",Govt.Gardan Road,635601.0,NaN,Primary Health Centre,Tamil Nadu,Vellore,Tirupathur,Tirupattur MPTY,12.492635,78.564177,382.6100
1,78.535451,12.266359,2,UTHANGARAI,"Government Hospital, Uthangarai",Uthangarai,Near Roundana,Uthangarai,635207.0,04341-222043,Sub-District Hospital,Tamil Nadu,Krishnagiri,Uthangarai,Uthangarai,12.266359,78.535451,343.4900
2,78.511082,11.045766,3,JAYANTHI,NO. 16/4,GANTHI NAGAR,"THRAIYUR ROAD, MUSIRI","THRAIYUR ROAD, MUSIRI",621211.0,04326 -260423,Others,Tamil Nadu,Tiruchirappalli,Musiri,Musiri,11.045766,78.511082,123.5000
3,78.559638,11.193647,4,KUMARAN,"NO. 120, TRICHY ROAD,TURAIYUR","TRICHY ROAD,TURAIYUR",TURAIYUR,TURAIYUR,621010.0,"+(91)-4327-2222343,???ÿ2222313",Others,Tamil Nadu,Tiruchirappalli,Thuraiyur,Thuraiyur MPTY,11.193647,78.559638,154.3000
4,78.526689,11.439301,5,Sendarapatti,HEALTH SUB CENTRE,AD STREET,PANCHAYAT OFFICE NEAR,SENDARAPATTY,636110.0,NaN,SubCentre,Tamil Nadu,Salem,Gangavalli,Gangavalli,11.439301,78.526689,284.5306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165106,76.465500,23.202200,165107,SHC Chakrod,NaN,kalapipal Road,Near Govt. high School,Chakrod,465333.0,NaN,SubCentre,Madhya Pradesh,Shajapur,Shujalpur,Shujalpur(Polaykalan),23.202200,76.465500,0.0000
165107,78.916286,23.120122,165108,PHC Dobhi,PHC,NaN,Village Dobhi,Tahsil Tendukheda Distt Narsinghpur,487337.0,NaN,Primary Health Centre,Madhya Pradesh,Narsinghpur,Tendukheda,Chawarpatha,23.120122,78.916286,0.0000
165108,79.905277,23.202764,165109,SHC Karmeta,KARMETA,KARMETA,KARMETA,KARMETA,482001.0,9479883739,SubCentre,Madhya Pradesh,Jabalpur,Panagar,Panagar,23.202764,79.905277,0.0000
165109,80.002659,23.120099,165110,SHC Umaria,NIL,Umaria kudari road,Hanumaan Mandir,Rural,483001.0,NaN,SubCentre,Madhya Pradesh,Jabalpur,Jabalpur,Jabalpur,23.120099,80.002659,0.0000


In [2]:
#get lat long using location name

from geopy.geocoders import Nominatim
def getlatlong(location_str):
    # Create a Nominatim geocoder object
    geolocator = Nominatim(user_agent="http")

    # Use geocoder to convert location string to latitude and longitude
    location = geolocator.geocode(location_str)


    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
        latitude=float(latitude)
        longitude=float(longitude)
    else:
        latitude,longitude=None,None
        
    return latitude,longitude



In [3]:
#get users current location

import geocoder

def get_live_location():
    g = geocoder.ip('me')
    return g.latlng

In [4]:
#calculate road distance between two locations

from geopy.distance import geodesic

def calculate_road_distance(lat1, lon1, lat2, lon2):
    # Coordinates of the two locations
    location1 = (lat1, lon1)
    location2 = (lat2, lon2)
    
    # Calculate the road distance between the two locations using geodesic distance
    distance = geodesic(location1, location2).kilometers
    
    return distance

In [5]:
#get users location

while True:
    a=int(input("enter 1 -to give your current location \n enter 2 -to type your location: "))
    if(a==1):
        latitude, longitude = get_live_location()
        print("Latitude:", latitude)
        print("Longitude:", longitude)
        break
    elif(a==2):
        a=input("enter location:")
        t=getlatlong(a)
        latitude=t[0]
        longitude=t[1]
        print("Latitude:", latitude)
        print("Longitude:", longitude)
        break
    else:
        print("choose correct option")
    


enter 1 -to give your current location 
 enter 2 -to type your location: 1
Latitude: 17.384
Longitude: 78.4564


In [6]:
n=int(input("no. of nearest hospitals u want:"))

no. of nearest hospitals u want:6


In [7]:
from sklearn.neighbors import NearestNeighbors



X = hospital_data[['lat', 'lon']].values
nbrs = NearestNeighbors(n_neighbors=n, algorithm='ball_tree').fit(X)


def suggest_nearest_hospitals(user_latitude, user_longitude):

    # Find the indices of the nearest hospitals based on the user's location
    distances, indices = nbrs.kneighbors([[user_latitude, user_longitude]])
    
    nearest_hospitals = []
    for index in indices[0]:
        hospital_name = hospital_data.loc[index, "health_facility_name"]
        hospital_latitude = hospital_data.loc[index, "lat"]
        hospital_longitude = hospital_data.loc[index, "lon"]
        hospital_distance = calculate_road_distance(user_latitude,user_longitude, hospital_latitude, hospital_longitude)
        nearest_hospitals.append((hospital_name, hospital_distance, hospital_latitude, hospital_longitude))
    
    return nearest_hospitals


# Suggest nearest hospitals if the location is within India
nearest_hospitals = []
#giving user lat-long to ml model
nearest_hospitals = suggest_nearest_hospitals(latitude, longitude)

# Print the results if there are nearest hospitals
if nearest_hospitals:
    print("Nearest Hospitals:")
    for hospital in nearest_hospitals:
        print(f"{hospital[0]} - Distance: {hospital[1]} km - (lat: {hospital[2]}, ;long: {hospital[3]})")
else:
    print("No nearest hospitals to display.")




Nearest Hospitals:
Urban Primary Health Centre, Afzal Sagar - Distance: 0.41243249227684 km - (lat: 17.3860234, ;long: 78.4596592)
Urban Primary Health Centre, Kummarwadi - Distance: 0.513729215368323 km - (lat: 17.3871135, ;long: 78.4528144)
MNJ Institute of Oncology and Regional Cancer Center - Distance: 0.9976561839445546 km - (lat: 17.39224327, ;long: 78.46019939)
Urban Primary Health Centre, Aghapura - Distance: 0.9796985612036072 km - (lat: 17.38816, ;long: 78.4645379)
Area Hospital, Nampally - Distance: 1.1726493532437248 km - (lat: 17.3922731, ;long: 78.4632945)
Urban Primary Health Centre, Nampally - Distance: 1.1726493532437248 km - (lat: 17.3922731, ;long: 78.4632945)


In [8]:
# Create a map centered around the user's location

import folium

map_center = (latitude, longitude)
m = folium.Map(location=map_center, zoom_start=12)

# Mark the user's location on the map
folium.Marker(
    location=map_center,
    popup="Your Location",
    icon=folium.Icon(color='blue')
).add_to(m)

# Mark the nearest hospitals on the map
for hospital in nearest_hospitals:
    hospital_name = hospital[0]
    hospital_distance=hospital[1]
    hospital_latitude = hospital[2]
    hospital_longitude = hospital[3]
    hospital_location = (hospital_latitude, hospital_longitude)

    folium.Marker(
        location=hospital_location,
        popup=f"{hospital_name} - Distance: {hospital[1]} km - lat: {hospital[2]}, ;long: {hospital[3]}",
        icon=folium.Icon(color='green', icon='info-sign')
    ).add_to(m)

# show on map
m